## 세티즌 데이터 확인용
#### 웨어러블 크롤링 - 이름, 제품코드, 설명, 상태, 가격, 시간, url, 판매여부
##### 0420

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'}

wearable = []
name = []
product_code = []
memo = []
condition = []
price = []
upload_time = []
sold = []
url_info = []
site = []
category = []



#마지막 페이지까지 크롤링 반복
for i in range(1,11):
    url = f"https://market.cetizen.com/market.php?p=1&keyword_p=&pno=&just_one=&just_one_name=&product_no=&pw=&pm=&m_no=&p_title=&pname=&p_key=&p_modelno=&p_model=&storage=&p_cat=&pm_modal=&series_modal=&datamode=&q=&at=&section=&cat%5B3%5D=10&price_range=0%3B5000000&auc_price1=0&auc_price2=5%2C000%2C000&storage_range=0%3B1024&storage1=0&storage2=1024&escrow_motion=3&select_arrange=headnum&desc=+&remove_dup=&list_type=1&p={i}"
    r = requests.get(url, headers=headers)
    r.status_code

    soup = BeautifulSoup(r.text,'html.parser')


    # 한페이지 정보 가져와 smartphone에 저장
    wearable = soup.find_all('div', class_= 'col-10')
    wearable = wearable[0::2]

    
    # 각 정보 해당 리스트에 저장하기
    for info in wearable:
        
        # 05/09 이후 데이터만 수집
        time_info = info.find("span", class_='gray2').text
        if (time_info == '06-06') : break
        
        # 상품 이름 
        name_info = info.find("span", class_='fsize6').text
        name.append(name_info)
    
        # 상품 코드 
        code_info = info.find("span", class_='ls-0').text
        product_code.append(code_info)
        
        # 상품 설명 
        memo_info = info.find("span", class_='pointer').text
        memo.append(memo_info)
        
        # 상품 상태 
        condition_info = info.find(['div','span'], class_= ['box_noicon_red', 'box_noicon_on']).text
        if condition_info == '새제품' : condition_info = '상'
        elif condition_info =='상' : condition_info = '중'
        else : condition_info = '하'
        condition.append(condition_info)

        # 가격 정보는 삭제된(==거래완료된) 정보일 경우와 아닌 경우로 구분하여 저장
        if ( info.select("span > del") ) : 
            price.append(info.find("del").text)
        elif ( info.find("span", class_='black0')) : 
            price.append(info.find("span", class_='black0').text)
        else : 
            price.append( info.find_all("span", class_='gray1')[1].text )
        
        # 업로드 시간 (시간으로 표시된 정보는 오늘 날짜로 저장)
        time_info = info.find("span", class_='gray2').text
        if(':' in time_info) : 
            upload_time.append('2022-06-09')
        elif('전' in time_info) : 
            upload_time.append('2022-06-02')
        else : upload_time.append(time_info)
        
        # 거래 완료 
        if ( info.find("span" , class_= "ml-2")) : 
            sold.append("거래완료")
        else : sold.append("")
            
        # url
        url_add = 'https://market.cetizen.com/' + info.find('a')['href']
        url_info.append(url_add)
            
        # 사이트명과 카테고리
        site.append("cetizen")
        category.append("웨어러블")

        
    time.sleep(1)


In [4]:
# DataFrame 생성 및 확인

df = pd.DataFrame(
        {"사이트" : site, 
         "카테고리" : category,
         "판매여부" : sold,
         "이름" : name,
         "제품코드" : product_code,
         "상태" : condition,
         "가격" : price,
         "업로드시간" : upload_time,
         "설명" : memo,
         "url" : url_info
        }
    )

df.shape
df


,사이트,카테고리,판매여부,이름,제품코드,상태,가격,업로드시간,설명,url
0,cetizen,웨어러블,,갤럭시워치4 44mm,SM-R870N,하,"180,000",2022-06-02,갤럭시 워치4 44mm 블루투스,https://market.cetizen.com//market.php?q=view&...
1,cetizen,웨어러블,,갤럭시워치4 40mm,SM-R860N,하,"160,000",2022-06-02,갤럭시 워치4 40mm 블루투스,https://market.cetizen.com//market.php?q=view&...
2,cetizen,웨어러블,,갤럭시워치4 40mm 셀룰러,SM-R865N,상,"170,000",2022-06-02,갤럭시워치4 40mm셀룰러 블랙,https://market.cetizen.com//market.php?q=view&...
3,cetizen,웨어러블,,갤럭시 워치 액티브2,SM-R830NZDAKOO,하,"110,000",2022-06-02,갤럭시 와치 액티브2 실버 4GB,https://market.cetizen.com//market.php?q=view&...
4,cetizen,웨어러블,,갤럭시 워치 46mm 셀룰러,SM-R805N,하,"90,000",2022-06-02,갤럭시워치/46MM/블랙/A급/셀룰러 판매합니다~!,https://market.cetizen.com//market.php?q=view&...
...,...,...,...,...,...,...,...,...,...,...
205,cetizen,웨어러블,,갤럭시워치4 40mm 셀룰러,SM-R865N,상,"170,000",05-27,워치4 실버 미개봉,https://market.cetizen.com//market.php?q=view&...
206,cetizen,웨어러블,,갤럭시워치4 40mm 셀룰러,SM-R865N,상,"170,000",05-27,워치4 실버 미개봉,https://market.cetizen.com//market.php?q=view&...
207,cetizen,웨어러블,거래완료,애플워치 시리즈 6 LTE 알루미늄 44mm,A2376,상,"430,000",05-27,애플워치 시리즈6 스페이스그레이 44mm 새상품 팔아요~,https://market.cetizen.com//market.php?q=view&...
208,cetizen,웨어러블,거래완료,갤럭시워치4 클래식 42mm 셀룰러,SM-R885N,상,"189,500",05-26,갤럭시워치4 클래식 42mm (R885) 블랙 미개봉 풀박스 판매,https://market.cetizen.com//market.php?q=view&...


In [5]:
# df.to_csv('cetizendata_wearable', index=False)
df.to_excel("cetizendata_wearable_0609.xlsx")